In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import catboost

In [ ]:
import tensorflow as tf
print(tf.test.is_gpu_available())

In [ ]:
# import tqdm

In [ ]:
import os
# basePath = os.path.dirname(os.path.abspath(__file__))
df=pd.read_json('../input/whats-cooking-kernels-only/train.json', orient='records', dtype={"id":int, "cuisine":str,"ingredients":list})

In [ ]:
df.head()

In [ ]:
df.shape

# посмотрим кол-во уникальных ингредиентов 

In [ ]:
lists = [df['ingredients'].values[i] for i in range(len(df))]

In [ ]:
unique_ingredients = list(set(list(np.concatenate(lists))))

In [ ]:
len(unique_ingredients)

In [ ]:
d = dict(zip(np.arange(len(unique_ingredients)), unique_ingredients))

In [ ]:
inv_d = dict(zip(unique_ingredients, np.arange(len(unique_ingredients))))

In [ ]:
inv_d

In [ ]:
X = np.zeros(shape = (len(df), len(unique_ingredients)))
for i in range(len(df)):
    if i % 10000 == 0:
        print(i)
    sample = df.loc[i]
    l_ingr = sample['ingredients']
    for ingr in l_ingr:
        X[i,inv_d[ingr]] = 1.0 

In [ ]:
np.sum(X, axis=1)

In [ ]:
y = df['cuisine'].values

In [ ]:
len(np.unique(y))

In [ ]:
df_test=pd.read_json('../input/whats-cooking-kernels-only/test.json', orient='records', dtype={"id":int,"ingredients":list})

In [ ]:
X_test = np.zeros(shape = (len(df_test), len(unique_ingredients)))
for i in range(len(df_test)):
    if i % 1000 == 0:
        print(i)
    sample = df_test.loc[i]
    l_ingr = sample['ingredients']
    for ingr in l_ingr:
        try:
          X_test[i,inv_d[ingr]] = 1.0 
        except:
          pass

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# clf = RandomForestClassifier(n_estimators = 100, max_depth=12, random_state=0, n_jobs=-1, verbose=True)
# clf.fit(X,y)
# y_test=clf.predict(X_test)

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Fit model
clf = MLPClassifier(solver='adam',activation='relu',early_stopping=True, random_state=3, 
                    max_iter=500,verbose=True,alpha=1e-08,beta_1=0.05, beta_2=0.4, epsilon=1e-09).fit(X, y)

In [ ]:
# model.fit(train_dataset, verbose=True)
# Get predicted classes
# preds_class = model.predict(eval_dataset)


In [ ]:
# model.score(eval_dataset)

In [ ]:
Y_test =clf.predict(X_test)

In [ ]:
Y_test = Y_test.reshape(-1)

In [ ]:
subm = pd.DataFrame({'id':df_test['id'], 'cuisine':Y_test})

In [ ]:
subm.to_csv('submission.csv',  index=False)

In [ ]:
subm